In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
import warnings
warnings.filterwarnings("ignore")
import requests
from bs4 import BeautifulSoup
import time

# Data Analysis Jobs

In [3]:
url_da = "https://www.linkedin.com/jobs/search/?currentJobId=3168594862&geoId=102478259&keywords=data%20analysis"

In [4]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

locations = []
titles = []
countries = []    
links = []
company_name = []
job_description = []
job_requirement = []
work_types = []

# Loop untuk scraping daftar pekerjaan
for i in range(0, 100, 25):
    url = url_da + "&start=" + str(i)
    request = requests.get(url)
    soup = BeautifulSoup(request.text, 'html.parser')

    # Location
    location = soup.findAll("span", {"class": "job-search-card__location"})
    for loc in location:
        locations.append(loc.text.replace("\n", " ").strip())

    # Title
    title = soup.findAll("h3", {"class": "base-search-card__title"})
    for t in title:
        titles.append(t.text.replace("\n", " ").strip())

    # Country
    for c in location:
        countries.append(c.text.replace("\n", " ").strip().split(',')[-1])

    # Job links
    job_links = soup.findAll("a", {"class": "base-card__full-link"})
    for link in job_links:
        links.append(link["href"])

    # Company names
    names = soup.findAll("a", {"class": "hidden-nested-link"})
    for name in names:
        company_name.append(name.text.replace("\n", " ").strip())

# Keywords untuk membagi deskripsi dan requirements
desc_keywords = ['responsibilities', 'responsibilities:', 'what you’ll do', 'what you will do', 'deskripsi', 'tugas', 'role', 'job description:', 'job description', 
                 'your main duties in flying with us', 'your impact', "you’ll get to"]
req_keywords = ['requirement', 'requirement:', 'requirements', 'requirements:', 'qualification', 'qualification:', 'qualifications', 'qualifications:', 'kualifikasi',
                'what you’ll need', 'what you will need', 'you should have', 'skills', 'experience', 'job requirements', 'mandatory belongings that you must prepare', 
                'who we look for', "you’ll need to succeed", 'what this role needs', 'kriteria']

# Loop untuk scraping detail tiap job
for link in links:
    try:
        job_page = requests.get(link)
        job_soup = BeautifulSoup(job_page.text, 'html.parser')

        # --- Ekstraksi Work Type (Fix sesuai struktur HTML) ---
        work_type_found = "Not Found"
        button = job_soup.find("button", class_="job-details-preferences-and-skills")
        if button:
            span_aria = button.find_all("span", attrs={"aria-hidden": "true"})
            for span in span_aria:
                text = span.get_text(strip=True)
                for tipe in ["Remote", "Hybrid", "On-site"]:
                    if tipe.lower() in text.lower():
                        work_type_found = tipe
                        break
        work_types.append(work_type_found)

        # --- Ekstraksi Job Description & Requirements ---
        desc_block = job_soup.find("div", {"class": "show-more-less-html__markup"})
        if desc_block:
            items = desc_block.find_all("li")
            li_texts = [li.text.strip() for li in items]

            desc_part = []
            req_part = []

            for li_text in li_texts:
                text_lower = li_text.lower()
                if any(k in text_lower for k in req_keywords):
                    req_part.append(li_text)
                elif any(k in text_lower for k in desc_keywords):
                    desc_part.append(li_text)
                else:
                    if any(word in text_lower for word in ['analyze', 'develop', 'manage', 'design', 'plan']):
                        desc_part.append(li_text)
                    else:
                        req_part.append(li_text)

            job_description.append(" | ".join(desc_part) if desc_part else "Not Found")
            job_requirement.append(" | ".join(req_part) if req_part else "Not Found")
        else:
            job_description.append("Not Found")
            job_requirement.append("Not Found")

        time.sleep(1)
    except Exception as e:
        job_description.append("Error: " + str(e))
        job_requirement.append("Error: " + str(e))
        work_types.append("Error")

In [5]:
df1 = pd.DataFrame({
    "Title": titles,
    "Company": company_name,
    "Location": locations,
    "Country": countries,
    "Job Description": job_description,
    "Job Requirements": job_requirement,
    "Work Type" : work_types,
    "Link": links
})

df1

,Title,Company,Location,Country,Job Description,Job Requirements,Work Type,Link
0,"Data Analyst, Business Intelligence",Shopee,"Jakarta, Indonesia",Indonesia,Assist the various business units as a discuss...,Serve as in-consultant who simplifies complex ...,Not Found,https://id.linkedin.com/jobs/view/data-analyst...
1,Business Analyst - SPX Express (Jakarta),Shopee,"Jakarta, Indonesia",Indonesia,Not Found,Perform deep data analysis to find pattern of ...,Not Found,https://id.linkedin.com/jobs/view/business-ana...
2,Data Analyst Marketing,PT ASTRA OTOPARTS Tbk,"Jakarta, Jakarta, Indonesia",Indonesia,Not Found,"Analysis about domestic performance, survey, r...",Not Found,https://id.linkedin.com/jobs/view/data-analyst...
3,Business Analyst,OTO,"Jakarta, Indonesia",Indonesia,Not Found,"Collecting, analyzing and interpreting data, a...",Not Found,https://id.linkedin.com/jobs/view/business-ana...
4,Wholesale Risk Analyst,PT. Bank Tabungan Negara (Persero) Tbk,"Jakarta, Indonesia",Indonesia,Not Found,Not Found,Not Found,https://id.linkedin.com/jobs/view/wholesale-ri...
...,...,...,...,...,...,...,...,...
235,Junior Data Analyst,PT Smart Multi Finance,"Serpong, Banten, Indonesia",Indonesia,Not Found,"Minimal S1 di bidang terkait, seperti: Statist...",Not Found,https://id.linkedin.com/jobs/view/junior-data-...
236,Customer Care Agent,Wings Group Indonesia (Sayap Mas Utama),"East Jakarta, Jakarta, Indonesia",Indonesia,Melakukan survey pelayanan dan mengusulkan act...,Melayani pertanyaan dan keluhan customers terh...,Not Found,https://id.linkedin.com/jobs/view/customer-car...
237,Management Trainee,PT Astra International Tbk,"Jakarta, Jakarta, Indonesia",Indonesia,Not Found,Gelar sarjana dari jurusan apa pun | Lulusan b...,Not Found,https://id.linkedin.com/jobs/view/management-t...
238,Performance Specialist,PT Alam Sutera Realty Tbk.,"Tangerang, Banten, Indonesia",Indonesia,Not Found,Bachelor's Degree majoring in Communication Sc...,Not Found,https://id.linkedin.com/jobs/view/performance-...


# Machine learning jobs

In [6]:
url_ml = "https://www.linkedin.com/jobs/search/?currentJobId=3168594862&geoId=102478259&keywords=machine%20learning&refresh=true"

In [7]:
locations = []
titles = []
countries = []    
links = []
company_name = []
job_description = []
job_requirement = []
work_types = []

for i in range(0, 100, 25):
    url = url_ml + "&start=" + str(i)
    request = requests.get(url)
    soup = BeautifulSoup(request.text, 'html.parser')

    # Location
    location = soup.findAll("span", {"class": "job-search-card__location"})
    for loc in location:
        locations.append(loc.text.replace("\n", " ").strip())

    # Title
    title = soup.findAll("h3", {"class": "base-search-card__title"})
    for t in title:
        titles.append(t.text.replace("\n", " ").strip())

    # Country
    for c in location:
        countries.append(c.text.replace("\n", " ").strip().split(',')[-1])

    # Job links
    job_links = soup.findAll("a", {"class": "base-card__full-link"})
    for link in job_links:
        links.append(link["href"])

    # Company names
    names = soup.findAll("a", {"class": "hidden-nested-link"})
    for name in names:
        company_name.append(name.text.replace("\n", " ").strip())

# Keywords
desc_keywords = ['responsibilities', 'responsibilities:', 'what you’ll do', 'what you will do', 'deskripsi', 'tugas', 'role', 'job description:', 'job description', 
                 'your main duties in flying with us', 'your impact', "you’ll get to"]
req_keywords = ['requirement', 'requirement:', 'requirements', 'requirements:', 'qualification', 'qualification:', 'qualifications', 'qualifications:', 'kualifikasi',
                'what you’ll need', 'what you will need', 'you should have', 'skills', 'experience', 'job requirements', 'mandatory belongings that you must prepare', 
                'who we look for', "you’ll need to succeed", 'what this role needs', 'kriteria']

for link in links:
    try:
        job_page = requests.get(link)
        job_soup = BeautifulSoup(job_page.text, 'html.parser')

        # Extract work type
        work_type_found = "Not Found"
        preference_button = job_soup.find("button", class_="job-details-preferences-and-skills")
        if preference_button:
            pill_divs = preference_button.find_all("div", class_="job-details-preferences-and-skills__pill")
            for pill in pill_divs:
                span = pill.find("span", class_="ui-label ui-label--accent-3 text-body-small")
                if span and span.text.strip() in ["Remote", "On-site", "Hybrid"]:
                    work_type_found = span.text.strip()
                    break
        work_types.append(work_type_found)

        # Extract job description & requirements
        desc_block = job_soup.find("div", {"class": "show-more-less-html__markup"})
        if desc_block:
            items = desc_block.find_all("li")
            li_texts = [li.text.strip() for li in items]

            desc_part = []
            req_part = []

            for li_text in li_texts:
                text_lower = li_text.lower()
                
                if any(k in text_lower for k in req_keywords):
                    req_part.append(li_text)
                elif any(k in text_lower for k in desc_keywords):
                    desc_part.append(li_text)
                else:
                    if any(word in text_lower for word in ['analyze', 'develop', 'manage', 'design', 'plan']):
                        desc_part.append(li_text)
                    else:
                        req_part.append(li_text)

            job_description.append(" | ".join(desc_part) if desc_part else "Not Found")
            job_requirement.append(" | ".join(req_part) if req_part else "Not Found")
        else:
            job_description.append("Not Found")
            job_requirement.append("Not Found")

        time.sleep(1)
    except Exception as e:
        job_description.append("Error: " + str(e))
        job_requirement.append("Error: " + str(e))
        work_types.append("Error")

In [8]:
df2 = pd.DataFrame({
    "Title": titles,
    "Company": company_name,
    "Location": locations,
    "Country": countries,
    "Job Description": job_description,
    "Job Requirements": job_requirement,
    "Work Type" : work_types,
    "Link": links
})

df2

,Title,Company,Location,Country,Job Description,Job Requirements,Work Type,Link
0,Machine Learning Engineer,INDICO by Telkomsel,"Jakarta, Jakarta, Indonesia",Indonesia,Design and implement machine learning models a...,Conduct experiments to optimize model performa...,Not Found,https://id.linkedin.com/jobs/view/machine-lear...
1,"Data Scientist, Business Intelligence",Shopee,"Jakarta, Indonesia",Indonesia,"Prototyping, developing, and deploying advance...",Communicating with the product and business te...,Not Found,https://id.linkedin.com/jobs/view/data-scienti...
2,"Data Analyst, Business Intelligence",Shopee,"Jakarta, Indonesia",Indonesia,Assist the various business units as a discuss...,Serve as in-consultant who simplifies complex ...,Not Found,https://id.linkedin.com/jobs/view/data-analyst...
3,AI/Machine Learning (Software Developer) (ASAP),SIGMATECH,"Jakarta, Jakarta, Indonesia",Indonesia,"Menguasai backend development dengan Flutter, ...",Menguasai bahasa pemrograman Python untuk trai...,Not Found,https://id.linkedin.com/jobs/view/ai-machine-l...
4,Data Science Intern,tiket.com,"Jakarta, Jakarta, Indonesia",Indonesia,Involve in model development using machine lea...,Conduct exploratory data analysis and grab ins...,Not Found,https://id.linkedin.com/jobs/view/data-science...
...,...,...,...,...,...,...,...,...
235,Machine Learning Manager (Indonesia),BJAK,Indonesia,Indonesia,"Design, develop, and implement AI models, algo...","Lead and mentor a team of AI engineers, provid...",Not Found,https://id.linkedin.com/jobs/view/machine-lear...
236,Data Scientist,Kredivo Group,Jakarta Metropolitan Area,Jakarta Metropolitan Area,Researching & Development on how to improve ou...,Working in a multi-disciplined team where you’...,Not Found,https://id.linkedin.com/jobs/view/data-scienti...
237,Analytical Junior Research Assistant Microbiology,Dexa Group,"North Cikarang, West Java, Indonesia",Indonesia,Not Found,Melakukan pengujian sampel stabilita dan memba...,Not Found,https://id.linkedin.com/jobs/view/analytical-j...
238,AI/ML Senior Software Engineer (Indonesia),BJAK,Indonesia,Indonesia,"Design, develop, and implement AI models, algo...",Work with product managers and stakeholders to...,Not Found,https://id.linkedin.com/jobs/view/ai-ml-senior...


# Software testing jobs

In [9]:
url_testing = "https://www.linkedin.com/jobs/search/?currentJobId=3251187123&geoId=102478259&keywords=software%20tester&refresh=true"

In [10]:
locations = []
titles = []
countries = []    
links = []
company_name = []
job_description = []
job_requirement = []
work_types = []

for i in range(0, 100, 25):
    url = url_testing + "&start=" + str(i)
    request = requests.get(url)
    soup = BeautifulSoup(request.text, 'html.parser')

    # Location
    location = soup.findAll("span", {"class": "job-search-card__location"})
    for loc in location:
        locations.append(loc.text.replace("\n", " ").strip())

    # Title
    title = soup.findAll("h3", {"class": "base-search-card__title"})
    for t in title:
        titles.append(t.text.replace("\n", " ").strip())

    # Country
    for c in location:
        countries.append(c.text.replace("\n", " ").strip().split(',')[-1])

    # Job links
    job_links = soup.findAll("a", {"class": "base-card__full-link"})
    for link in job_links:
        links.append(link["href"])

    # Company names
    names = soup.findAll("a", {"class": "hidden-nested-link"})
    for name in names:
        company_name.append(name.text.replace("\n", " ").strip())

# Keywords
desc_keywords = ['responsibilities', 'responsibilities:', 'what you’ll do', 'what you will do', 'deskripsi', 'tugas', 'role', 'job description:', 'job description', 
                 'your main duties in flying with us', 'your impact', "you’ll get to"]
req_keywords = ['requirement', 'requirement:', 'requirements', 'requirements:', 'qualification', 'qualification:', 'qualifications', 'qualifications:', 'kualifikasi',
                'what you’ll need', 'what you will need', 'you should have', 'skills', 'experience', 'job requirements', 'mandatory belongings that you must prepare', 
                'who we look for', "you’ll need to succeed", 'what this role needs', 'kriteria']

for link in links:
    try:
        job_page = requests.get(link)
        job_soup = BeautifulSoup(job_page.text, 'html.parser')

        # Extract work type
        work_type_found = "Not Found"
        preference_button = job_soup.find("button", class_="job-details-preferences-and-skills")
        if preference_button:
            pill_divs = preference_button.find_all("div", class_="job-details-preferences-and-skills__pill")
            for pill in pill_divs:
                span = pill.find("span", class_="ui-label ui-label--accent-3 text-body-small")
                if span and span.text.strip() in ["Remote", "On-site", "Hybrid"]:
                    work_type_found = span.text.strip()
                    break
        work_types.append(work_type_found)

        # Extract job description & requirements
        desc_block = job_soup.find("div", {"class": "show-more-less-html__markup"})
        if desc_block:
            items = desc_block.find_all("li")
            li_texts = [li.text.strip() for li in items]

            desc_part = []
            req_part = []

            for li_text in li_texts:
                text_lower = li_text.lower()
                
                if any(k in text_lower for k in req_keywords):
                    req_part.append(li_text)
                elif any(k in text_lower for k in desc_keywords):
                    desc_part.append(li_text)
                else:
                    if any(word in text_lower for word in ['analyze', 'develop', 'manage', 'design', 'plan']):
                        desc_part.append(li_text)
                    else:
                        req_part.append(li_text)

            job_description.append(" | ".join(desc_part) if desc_part else "Not Found")
            job_requirement.append(" | ".join(req_part) if req_part else "Not Found")
        else:
            job_description.append("Not Found")
            job_requirement.append("Not Found")

        time.sleep(1)
    except Exception as e:
        job_description.append("Error: " + str(e))
        job_requirement.append("Error: " + str(e))
        work_types.append("Error")

In [11]:
df3 = pd.DataFrame({
    "Title": titles,
    "Company": company_name,
    "Location": locations,
    "Country": countries,
    "Job Description": job_description,
    "Job Requirements": job_requirement,
    "Work Type" : work_types,
    "Link": links
})

df3

,Title,Company,Location,Country,Job Description,Job Requirements,Work Type,Link
0,QA Engineer (ShopeePay) - Sea Labs,Shopee,"Jakarta, Indonesia",Indonesia,Be involved in whole development life cycle an...,Functional testing via grey-box and white-box ...,Not Found,https://id.linkedin.com/jobs/view/qa-engineer-...
1,QA Engineer (Credit) - Sea Labs,Shopee,"Jakarta, Indonesia",Indonesia,Be involved in whole development life cycle an...,Functional testing via grey-box and white-box ...,Not Found,https://id.linkedin.com/jobs/view/qa-engineer-...
2,Quality Assurance Engineer,Traveloka,Jakarta Metropolitan Area,Jakarta Metropolitan Area,"Work with Product Managers, Engineering team, ...",Create test plan and test cases based on the p...,Not Found,https://id.linkedin.com/jobs/view/quality-assu...
3,QA Tester (Manual),Jubelio,"Jakarta, Jakarta, Indonesia",Indonesia,Understand software Development Life Cycle,Create test case API positif and negative | Yo...,Not Found,https://id.linkedin.com/jobs/view/qa-tester-ma...
4,Quality Assurance Tester,PT. Lintas Teknologi Indonesia,"Depok, Yogyakarta, Indonesia",Indonesia,Not Found,Bachelor Degree is a MUST! | 2+ years experien...,Not Found,https://id.linkedin.com/jobs/view/quality-assu...
...,...,...,...,...,...,...,...,...
235,Expert QA Engineer (Credit) - Sea Labs,Shopee,"Jakarta, Indonesia",Indonesia,Manage the feature testing and/or regular regr...,Continuous improvement and research of testing...,Not Found,https://id.linkedin.com/jobs/view/expert-qa-en...
236,Expert QA Engineer (Credit) - Sea Labs,Sea,"Jakarta, Indonesia",Indonesia,Manage the feature testing and/or regular regr...,Continuous improvement and research of testing...,Not Found,https://id.linkedin.com/jobs/view/expert-qa-en...
237,QA Engineer,ZettaByte Pte Ltd,"Yogyakarta, Yogyakarta, Indonesia",Indonesia,Not Found,Diploma or Bachelor’s degree in Computer Scien...,Not Found,https://id.linkedin.com/jobs/view/qa-engineer-...
238,QA Engineer 2 (Mekari Jurnal),Mekari,"Jakarta, Jakarta, Indonesia",Indonesia,Not Found,"Define, describe, review test cases for all re...",Not Found,https://id.linkedin.com/jobs/view/qa-engineer-...


# Fullstack Developer

In [12]:
url_fullstack = "https://www.linkedin.com/jobs/search/?currentJobId=4227359993&geoId=102478259&keywords=fullstack%20developer&refresh=true"

In [13]:
locations = []
titles = []
countries = []    
links = []
company_name = []
job_description = []
job_requirement = []
work_types = []

for i in range(0, 100, 25):
    url = url_fullstack + "&start=" + str(i)
    request = requests.get(url)
    soup = BeautifulSoup(request.text, 'html.parser')

    # Location
    location = soup.findAll("span", {"class": "job-search-card__location"})
    for loc in location:
        locations.append(loc.text.replace("\n", " ").strip())

    # Title
    title = soup.findAll("h3", {"class": "base-search-card__title"})
    for t in title:
        titles.append(t.text.replace("\n", " ").strip())

    # Country
    for c in location:
        countries.append(c.text.replace("\n", " ").strip().split(',')[-1])

    # Job links
    job_links = soup.findAll("a", {"class": "base-card__full-link"})
    for link in job_links:
        links.append(link["href"])

    # Company names
    names = soup.findAll("a", {"class": "hidden-nested-link"})
    for name in names:
        company_name.append(name.text.replace("\n", " ").strip())

# Keywords
desc_keywords = ['responsibilities', 'responsibilities:', 'what you’ll do', 'what you will do', 'deskripsi', 'tugas', 'role', 'job description:', 'job description', 
                 'your main duties in flying with us', 'your impact', "you’ll get to"]
req_keywords = ['requirement', 'requirement:', 'requirements', 'requirements:', 'qualification', 'qualification:', 'qualifications', 'qualifications:', 'kualifikasi',
                'what you’ll need', 'what you will need', 'you should have', 'skills', 'experience', 'job requirements', 'mandatory belongings that you must prepare', 
                'who we look for', "you’ll need to succeed", 'what this role needs', 'kriteria']

for link in links:
    try:
        job_page = requests.get(link)
        job_soup = BeautifulSoup(job_page.text, 'html.parser')

        # Extract work type
        work_type_found = "Not Found"
        preference_button = job_soup.find("button", class_="job-details-preferences-and-skills")
        if preference_button:
            pill_divs = preference_button.find_all("div", class_="job-details-preferences-and-skills__pill")
            for pill in pill_divs:
                span = pill.find("span", class_="ui-label ui-label--accent-3 text-body-small")
                if span and span.text.strip() in ["Remote", "On-site", "Hybrid"]:
                    work_type_found = span.text.strip()
                    break
        work_types.append(work_type_found)

        # Extract job description & requirements
        desc_block = job_soup.find("div", {"class": "show-more-less-html__markup"})
        if desc_block:
            items = desc_block.find_all("li")
            li_texts = [li.text.strip() for li in items]

            desc_part = []
            req_part = []

            for li_text in li_texts:
                text_lower = li_text.lower()
                
                if any(k in text_lower for k in req_keywords):
                    req_part.append(li_text)
                elif any(k in text_lower for k in desc_keywords):
                    desc_part.append(li_text)
                else:
                    if any(word in text_lower for word in ['analyze', 'develop', 'manage', 'design', 'plan']):
                        desc_part.append(li_text)
                    else:
                        req_part.append(li_text)

            job_description.append(" | ".join(desc_part) if desc_part else "Not Found")
            job_requirement.append(" | ".join(req_part) if req_part else "Not Found")
        else:
            job_description.append("Not Found")
            job_requirement.append("Not Found")

        time.sleep(1)
    except Exception as e:
        job_description.append("Error: " + str(e))
        job_requirement.append("Error: " + str(e))
        work_types.append("Error")

In [14]:
min_len = min(len(titles), len(locations), len(countries), len(links), len(company_name), len(job_description), len(job_requirement))

In [15]:
titles = titles[:min_len]
locations = locations[:min_len]
countries = countries[:min_len]
links = links[:min_len]
company_name = company_name[:min_len]
job_description = job_description[:min_len]
job_requirement = job_requirement[:min_len]

In [16]:
df4 = pd.DataFrame({
    "Title": titles,
    "Company": company_name,
    "Location": locations,
    "Country": countries,
    "Job Description": job_description,
    "Job Requirements": job_requirement,
    "Work Type" : work_types,
    "Link": links
})

df4

,Title,Company,Location,Country,Job Description,Job Requirements,Work Type,Link
0,Fullstack Developer,PT Astra International Tbk,"Jakarta, Jakarta, Indonesia",Indonesia,"Membuat, memperbarui, dan memantau tugas yang ...",Membangun front-end web dan back-end API (fitu...,Not Found,https://id.linkedin.com/jobs/view/fullstack-de...
1,Application Software Developer,PT Sinergy Informasi Pratama,"Jakarta, Jakarta, Indonesia",Indonesia,1 hingga 2 tahun pengalaman sebagai Fullstack ...,"Min. S1 jurusan IT, Rekayasa Perangkat Lunak, ...",Not Found,https://id.linkedin.com/jobs/view/application-...
2,Fullstack Developer,PT Astra International Tbk,"Jakarta, Jakarta, Indonesia",Indonesia,Memahami Development di Azure Cloud Environment,"Merancang, mendesain, mengembangkan dan memodi...",Not Found,https://id.linkedin.com/jobs/view/fullstack-de...
3,Fullstack Developer (Site: Sumatera Selatan),PT. Metrodata Electronics Tbk.,"Jakarta, Jakarta, Indonesia",Indonesia,"Design, Develop, Analyze, troubleshooting rela...",Conduct analysis and problem solving of applic...,Not Found,https://id.linkedin.com/jobs/view/fullstack-de...
4,Software Engineer (Fullstack),Sprout Digital Labs,"Tangerang Regency, Banten, Indonesia",Indonesia,You will be deeply engaged in the full develop...,"Provide technical leadership in a fast-moving,...",Not Found,https://id.linkedin.com/jobs/view/software-eng...
...,...,...,...,...,...,...,...,...
223,UI Solution Engineer 1 (Mekari Officeless),Asiatek Solusi Indonesia,"Jakarta, Jakarta, Indonesia",Indonesia,Not Found,Back-End Web Development and Full-Stack Develo...,Not Found,https://id.linkedin.com/jobs/view/fullstack-de...
224,Remote Nextjs Fullstack Developer,Agnos Inc.,Indonesia,Indonesia,We work with our clients to revolutionize thei...,Our current client is focused on transforming ...,Not Found,https://id.linkedin.com/jobs/view/lead-softwar...
225,Java Developer Backend Developer,PT Hermes Solusi Integrasi,"Jakarta, Jakarta, Indonesia",Indonesia,Not Found,Experience and understand on developing busine...,Not Found,https://id.linkedin.com/jobs/view/senior-fulls...
226,Lead Software Engineer (RoR Fullstack Engineer...,PT. Digital Netwerk Venture Indonesia | The Ne...,Indonesia,Indonesia,Not Found,Not Found,Not Found,https://id.linkedin.com/jobs/view/full-stack-e...


# UI/UX

In [17]:
url_uiux = "https://www.linkedin.com/jobs/search/?currentJobId=4227359993&geoId=102478259&keywords=ui%20ux&refresh=true"

In [18]:
locations = []
titles = []
countries = []    
links = []
company_name = []
job_description = []
job_requirement = []
work_types = []

for i in range(0, 100, 25):
    url = url_uiux + "&start=" + str(i)
    request = requests.get(url)
    soup = BeautifulSoup(request.text, 'html.parser')

    # Location
    location = soup.findAll("span", {"class": "job-search-card__location"})
    for loc in location:
        locations.append(loc.text.replace("\n", " ").strip())

    # Title
    title = soup.findAll("h3", {"class": "base-search-card__title"})
    for t in title:
        titles.append(t.text.replace("\n", " ").strip())

    # Country
    for c in location:
        countries.append(c.text.replace("\n", " ").strip().split(',')[-1])

    # Job links
    job_links = soup.findAll("a", {"class": "base-card__full-link"})
    for link in job_links:
        links.append(link["href"])

    # Company names
    names = soup.findAll("a", {"class": "hidden-nested-link"})
    for name in names:
        company_name.append(name.text.replace("\n", " ").strip())

# Keywords
desc_keywords = ['responsibilities', 'responsibilities:', 'what you’ll do', 'what you will do', 'deskripsi', 'tugas', 'role', 'job description:', 'job description', 
                 'your main duties in flying with us', 'your impact', "you’ll get to"]
req_keywords = ['requirement', 'requirement:', 'requirements', 'requirements:', 'qualification', 'qualification:', 'qualifications', 'qualifications:', 'kualifikasi',
                'what you’ll need', 'what you will need', 'you should have', 'skills', 'experience', 'job requirements', 'mandatory belongings that you must prepare', 
                'who we look for', "you’ll need to succeed", 'what this role needs', 'kriteria']

for link in links:
    try:
        job_page = requests.get(link)
        job_soup = BeautifulSoup(job_page.text, 'html.parser')

        # Extract work type
        work_type_found = "Not Found"
        preference_button = job_soup.find("button", class_="job-details-preferences-and-skills")
        if preference_button:
            pill_divs = preference_button.find_all("div", class_="job-details-preferences-and-skills__pill")
            for pill in pill_divs:
                span = pill.find("span", class_="ui-label ui-label--accent-3 text-body-small")
                if span and span.text.strip() in ["Remote", "On-site", "Hybrid"]:
                    work_type_found = span.text.strip()
                    break
        work_types.append(work_type_found)

        # Extract job description & requirements
        desc_block = job_soup.find("div", {"class": "show-more-less-html__markup"})
        if desc_block:
            items = desc_block.find_all("li")
            li_texts = [li.text.strip() for li in items]

            desc_part = []
            req_part = []

            for li_text in li_texts:
                text_lower = li_text.lower()
                
                if any(k in text_lower for k in req_keywords):
                    req_part.append(li_text)
                elif any(k in text_lower for k in desc_keywords):
                    desc_part.append(li_text)
                else:
                    if any(word in text_lower for word in ['analyze', 'develop', 'manage', 'design', 'plan']):
                        desc_part.append(li_text)
                    else:
                        req_part.append(li_text)

            job_description.append(" | ".join(desc_part) if desc_part else "Not Found")
            job_requirement.append(" | ".join(req_part) if req_part else "Not Found")
        else:
            job_description.append("Not Found")
            job_requirement.append("Not Found")

        time.sleep(1)
    except Exception as e:
        job_description.append("Error: " + str(e))
        job_requirement.append("Error: " + str(e))
        work_types.append("Error")

In [19]:
df5 = pd.DataFrame({
    "Title": titles,
    "Company": company_name,
    "Location": locations,
    "Country": countries,
    "Job Description": job_description,
    "Job Requirements": job_requirement,
    "Work Type" : work_types,
    "Link": links
})

df5

,Title,Company,Location,Country,Job Description,Job Requirements,Work Type,Link
0,Product Intern,INDICO by Telkomsel,Jakarta Metropolitan Area,Jakarta Metropolitan Area,Assist the product team in product development...,Draft and assist in the creation of Product Re...,Not Found,https://id.linkedin.com/jobs/view/product-inte...
1,Digital Marketing Analyst,PT ASTRA OTOPARTS Tbk,"Jakarta, Jakarta, Indonesia",Indonesia,Not Found,Manage B2B Digtal Business in terms of Product...,Not Found,https://id.linkedin.com/jobs/view/digital-mark...
2,UI/UX Consultant,EY,"Jakarta, Jakarta, Indonesia",Indonesia,Provide design guideline standard to ensure go...,Provide guidance on the implementation of UX r...,Not Found,https://id.linkedin.com/jobs/view/ui-ux-consul...
3,UI/UX Designer,Sprout Digital Labs,"Tangerang, Banten, Indonesia",Indonesia,Develop understanding of the end users of the ...,Responsible for understanding project brief an...,Not Found,https://id.linkedin.com/jobs/view/ui-ux-design...
4,UI / UX Designer,PT Astra International Tbk,"Jakarta, Jakarta, Indonesia",Indonesia,Mengembangkan dan mendesign interface aplikasi...,Memiliki pengalaman 1-2 tahun di bidang terseb...,Not Found,https://id.linkedin.com/jobs/view/ui-ux-design...
...,...,...,...,...,...,...,...,...
235,Website Designer,Reel Unlimited,Indonesia,Indonesia,Design visually appealing and user-friendly we...,Create responsive layouts that work seamlessly...,Not Found,https://id.linkedin.com/jobs/view/website-desi...
236,UX Content Analyst,Mobbin,"Jakarta, Jakarta, Indonesia",Indonesia,Annotate screens and flows with information ab...,"Acquire and prepare content on a weekly basis,...",Not Found,https://id.linkedin.com/jobs/view/ux-content-a...
237,Jr. Product Manager,Synapsis,"Gambir, Jakarta, Indonesia",Indonesia,"Conduct in-depth research on user needs, pain ...",Define product requirements and propose new fe...,Not Found,https://id.linkedin.com/jobs/view/jr-product-m...
238,Senior UI/UX Designer,Mitramas Infosys Global,"Jakarta, Jakarta, Indonesia",Indonesia,"Develop the product’s look and feel, including...",Conduct research about website and app UI and ...,Not Found,https://id.linkedin.com/jobs/view/senior-ui-ux...


# Cybersecurity

In [20]:
url_cyber = "https://www.linkedin.com/jobs/search/?currentJobId=4215815843&geoId=102478259&keywords=cybersecurity&refresh=true"

In [21]:
locations = []
titles = []
countries = []    
links = []
company_name = []
job_description = []
job_requirement = []
work_types = []

for i in range(0, 100, 25):
    url = url_cyber + "&start=" + str(i)
    request = requests.get(url)
    soup = BeautifulSoup(request.text, 'html.parser')

    # Location
    location = soup.findAll("span", {"class": "job-search-card__location"})
    for loc in location:
        locations.append(loc.text.replace("\n", " ").strip())

    # Title
    title = soup.findAll("h3", {"class": "base-search-card__title"})
    for t in title:
        titles.append(t.text.replace("\n", " ").strip())

    # Country
    for c in location:
        countries.append(c.text.replace("\n", " ").strip().split(',')[-1])

    # Job links
    job_links = soup.findAll("a", {"class": "base-card__full-link"})
    for link in job_links:
        links.append(link["href"])

    # Company names
    names = soup.findAll("a", {"class": "hidden-nested-link"})
    for name in names:
        company_name.append(name.text.replace("\n", " ").strip())

# Keywords
desc_keywords = ['responsibilities', 'responsibilities:', 'what you’ll do', 'what you will do', 'deskripsi', 'tugas', 'role', 'job description:', 'job description', 
                 'your main duties in flying with us', 'your impact', "you’ll get to"]
req_keywords = ['requirement', 'requirement:', 'requirements', 'requirements:', 'qualification', 'qualification:', 'qualifications', 'qualifications:', 'kualifikasi',
                'what you’ll need', 'what you will need', 'you should have', 'skills', 'experience', 'job requirements', 'mandatory belongings that you must prepare', 
                'who we look for', "you’ll need to succeed", 'what this role needs', 'kriteria']

for link in links:
    try:
        job_page = requests.get(link)
        job_soup = BeautifulSoup(job_page.text, 'html.parser')

        # Extract work type
        work_type_found = "Not Found"
        preference_button = job_soup.find("button", class_="job-details-preferences-and-skills")
        if preference_button:
            pill_divs = preference_button.find_all("div", class_="job-details-preferences-and-skills__pill")
            for pill in pill_divs:
                span = pill.find("span", class_="ui-label ui-label--accent-3 text-body-small")
                if span and span.text.strip() in ["Remote", "On-site", "Hybrid"]:
                    work_type_found = span.text.strip()
                    break
        work_types.append(work_type_found)

        # Extract job description & requirements
        desc_block = job_soup.find("div", {"class": "show-more-less-html__markup"})
        if desc_block:
            items = desc_block.find_all("li")
            li_texts = [li.text.strip() for li in items]

            desc_part = []
            req_part = []

            for li_text in li_texts:
                text_lower = li_text.lower()
                
                if any(k in text_lower for k in req_keywords):
                    req_part.append(li_text)
                elif any(k in text_lower for k in desc_keywords):
                    desc_part.append(li_text)
                else:
                    if any(word in text_lower for word in ['analyze', 'develop', 'manage', 'design', 'plan']):
                        desc_part.append(li_text)
                    else:
                        req_part.append(li_text)

            job_description.append(" | ".join(desc_part) if desc_part else "Not Found")
            job_requirement.append(" | ".join(req_part) if req_part else "Not Found")
        else:
            job_description.append("Not Found")
            job_requirement.append("Not Found")

        time.sleep(1)
    except Exception as e:
        job_description.append("Error: " + str(e))
        job_requirement.append("Error: " + str(e))
        work_types.append("Error")

In [22]:
df6 = pd.DataFrame({
    "Title": titles,
    "Company": company_name,
    "Location": locations,
    "Country": countries,
    "Job Description": job_description,
    "Job Requirements": job_requirement,
    "Work Type" : work_types,
    "Link": links
})

df6

,Title,Company,Location,Country,Job Description,Job Requirements,Work Type,Link
0,Cybersecurity Junior Consultant,EY,"Jakarta, Jakarta, Indonesia",Indonesia,Cyber Engineer – Managed Services | Managed Se...,Cyber Engineer – Red Team and Blue Team | Cybe...,Not Found,https://id.linkedin.com/jobs/view/cybersecurit...
1,Corporate Security Intern,PT Astra International Tbk,"Jakarta, Jakarta, Indonesia",Indonesia,Not Found,"Menganalisa Sistem Keamanan, Pengembangan SDM,...",Not Found,https://id.linkedin.com/jobs/view/corporate-se...
2,IT Security,PT. Astra Graphia Information Technology (AGIT),"Jakarta, Indonesia",Indonesia,"Perform monitor, manage, and configure securit...",IT Security jaringan | Menguasai perangkat sec...,Not Found,https://id.linkedin.com/jobs/view/it-security-...
3,Information Technology Security Architect,PT Lion Super Indo,"South Jakarta, Jakarta, Indonesia",Indonesia,Security Awareness: Champion a culture of secu...,Business Collaboration: Partner with business ...,Not Found,https://id.linkedin.com/jobs/view/information-...
4,IT Security,OCBC Indonesia,"Tangerang, Banten, Indonesia",Indonesia,Architecting and designing cybersecurity solut...,As a Security Architect - Create and enhance s...,Not Found,https://id.linkedin.com/jobs/view/it-security-...
...,...,...,...,...,...,...,...,...
235,Penetration Test Engineer – Blue Team L2 (WFO ...,SIGMATECH,"Jakarta, Jakarta, Indonesia",Indonesia,"Collaborate with SOC, DevOps, and Infrastructu...","Perform regular penetration testing, vulnerabi...",Not Found,https://id.linkedin.com/jobs/view/penetration-...
236,Security Operation Manager,Traveloka,Jakarta Metropolitan Area,Jakarta Metropolitan Area,Manage Traveloka Security Operations team to r...,Provide expertise to help improve security dom...,Not Found,https://id.linkedin.com/jobs/view/security-ope...
237,IT Security Specialist,Kredit Pintar,"South Jakarta, Jakarta, Indonesia",Indonesia,"Investigate and analyze security alerts, ident...",Monitor security systems and tools to detect a...,Not Found,https://id.linkedin.com/jobs/view/it-security-...
238,Cyber Security Strategy Planning Head,PT Bank SMBC Indonesia Tbk,"Jakarta, Indonesia",Indonesia,"Coordinate with the Cyber Security Governance,...","In periodically and ad hoc basis, carry out th...",Not Found,https://id.linkedin.com/jobs/view/cyber-securi...


# Quality Assurance

In [23]:
url_qa = "https://www.linkedin.com/jobs/search/?currentJobId=4223685654&geoId=102478259&keywords=quality%20assurance&refresh=true"

In [24]:
locations = []
titles = []
countries = []    
links = []
company_name = []
job_description = []
job_requirement = []
work_types = []

for i in range(0, 100, 25):
    url = url_qa + "&start=" + str(i)
    request = requests.get(url)
    soup = BeautifulSoup(request.text, 'html.parser')

    # Location
    location = soup.findAll("span", {"class": "job-search-card__location"})
    for loc in location:
        locations.append(loc.text.replace("\n", " ").strip())

    # Title
    title = soup.findAll("h3", {"class": "base-search-card__title"})
    for t in title:
        titles.append(t.text.replace("\n", " ").strip())

    # Country
    for c in location:
        countries.append(c.text.replace("\n", " ").strip().split(',')[-1])

    # Job links
    job_links = soup.findAll("a", {"class": "base-card__full-link"})
    for link in job_links:
        links.append(link["href"])

    # Company names
    names = soup.findAll("a", {"class": "hidden-nested-link"})
    for name in names:
        company_name.append(name.text.replace("\n", " ").strip())

# Keywords
desc_keywords = ['responsibilities', 'responsibilities:', 'what you’ll do', 'what you will do', 'deskripsi', 'tugas', 'role', 'job description:', 'job description', 
                 'your main duties in flying with us', 'your impact', "you’ll get to"]
req_keywords = ['requirement', 'requirement:', 'requirements', 'requirements:', 'qualification', 'qualification:', 'qualifications', 'qualifications:', 'kualifikasi',
                'what you’ll need', 'what you will need', 'you should have', 'skills', 'experience', 'job requirements', 'mandatory belongings that you must prepare', 
                'who we look for', "you’ll need to succeed", 'what this role needs', 'kriteria']

for link in links:
    try:
        job_page = requests.get(link)
        job_soup = BeautifulSoup(job_page.text, 'html.parser')

        # Extract work type
        work_type_found = "Not Found"
        preference_button = job_soup.find("button", class_="job-details-preferences-and-skills")
        if preference_button:
            pill_divs = preference_button.find_all("div", class_="job-details-preferences-and-skills__pill")
            for pill in pill_divs:
                span = pill.find("span", class_="ui-label ui-label--accent-3 text-body-small")
                if span and span.text.strip() in ["Remote", "On-site", "Hybrid"]:
                    work_type_found = span.text.strip()
                    break
        work_types.append(work_type_found)

        # Extract job description & requirements
        desc_block = job_soup.find("div", {"class": "show-more-less-html__markup"})
        if desc_block:
            items = desc_block.find_all("li")
            li_texts = [li.text.strip() for li in items]

            desc_part = []
            req_part = []

            for li_text in li_texts:
                text_lower = li_text.lower()
                
                if any(k in text_lower for k in req_keywords):
                    req_part.append(li_text)
                elif any(k in text_lower for k in desc_keywords):
                    desc_part.append(li_text)
                else:
                    if any(word in text_lower for word in ['analyze', 'develop', 'manage', 'design', 'plan']):
                        desc_part.append(li_text)
                    else:
                        req_part.append(li_text)

            job_description.append(" | ".join(desc_part) if desc_part else "Not Found")
            job_requirement.append(" | ".join(req_part) if req_part else "Not Found")
        else:
            job_description.append("Not Found")
            job_requirement.append("Not Found")

        time.sleep(1)
    except Exception as e:
        job_description.append("Error: " + str(e))
        job_requirement.append("Error: " + str(e))
        work_types.append("Error")

In [25]:
min_len = min(len(titles), len(locations), len(countries), len(links), len(company_name), len(job_description), len(job_requirement))

In [26]:
titles = titles[:min_len]
locations = locations[:min_len]
countries = countries[:min_len]
links = links[:min_len]
company_name = company_name[:min_len]
job_description = job_description[:min_len]
job_requirement = job_requirement[:min_len]

In [27]:
df7 = pd.DataFrame({
    "Title": titles,
    "Company": company_name,
    "Location": locations,
    "Country": countries,
    "Job Description": job_description,
    "Job Requirements": job_requirement,
    "Work Type" : work_types,
    "Link": links
})

df7

,Title,Company,Location,Country,Job Description,Job Requirements,Work Type,Link
0,Quality Assurance & Regulatory Staff,Kalbe Nutritionals (PT Sanghiang Perkasa),"Jakarta, Jakarta, Indonesia",Indonesia,Manage incoming product lab testing to ensure ...,Register all products and checking the status ...,Not Found,https://id.linkedin.com/jobs/view/quality-assu...
1,QA Microbiology Jr. Staff,Kalbe Nutritionals (PT Sanghiang Perkasa),"West Karawang, West Java, Indonesia",Indonesia,Not Found,Melakukan analisa sampel patogen dan non patog...,Not Found,https://id.linkedin.com/jobs/view/qa-microbiol...
2,Packaging Development Staff (Maternity Leave),Kalbe Nutritionals (PT Sanghiang Perkasa),"East Jakarta, Jakarta, Indonesia",Indonesia,Memiliki pengalaman bidang Packaging Developme...,Mengetahui basic knowledge tentang teknologi p...,Not Found,https://id.linkedin.com/jobs/view/packaging-de...
3,Development and QA Analyst,PT Astra International Tbk,"Jakarta, Jakarta, Indonesia",Indonesia,Mendukung proses management reporting (analisa...,Melakukan monitoring atas aktivitas administra...,Not Found,https://id.linkedin.com/jobs/view/development-...
4,Quality Assurance (Customer Service) - SPX Exp...,Shopee,"Jakarta, Indonesia",Indonesia,Not Found,Check Claim Team ticket interactions and ticke...,Not Found,https://id.linkedin.com/jobs/view/quality-assu...
...,...,...,...,...,...,...,...,...
231,Quality Assurance (QA) ASSA \tPurwokerto,SIGMATECH,"Banyumas, Central Java, Indonesia",Indonesia,Familiar with Agile software development | Dev...,Experience with testing and performance test a...,Not Found,https://id.linkedin.com/jobs/view/qa-manual-mi...
232,Warehouse - Quality Assurance (QA) - Lampung,Bibit.id,Greater Bandar Lampung,Greater Bandar Lampung,Define and analyze metrics to guide product de...,"Understand markets, industry landscapes, and u...",Not Found,https://id.linkedin.com/jobs/view/product-mana...
233,Quality Assurance Data Analyst,Sea,"Jakarta, Jakarta, Indonesia",Indonesia,Be involved in whole development life cycle an...,Functional testing via grey-box and white-box ...,Not Found,https://id.linkedin.com/jobs/view/qa-engineer-...
234,Warehouse - Quality Assurance (QA) - Batam,Anteraja,"Batam, Riau Islands, Indonesia",Indonesia,Not Found,Melakukan pengawasan terhadap proses harian op...,Not Found,https://id.linkedin.com/jobs/view/warehouse-qu...


# DevOps

In [28]:
url_devops = "https://www.linkedin.com/jobs/search/?currentJobId=4218023799&geoId=102478259&keywords=devops&refresh=true"

In [29]:
locations = []
titles = []
countries = []    
links = []
company_name = []
job_description = []
job_requirement = []
work_types = []

for i in range(0, 100, 25):
    url = url_devops + "&start=" + str(i)
    request = requests.get(url)
    soup = BeautifulSoup(request.text, 'html.parser')

    # Location
    location = soup.findAll("span", {"class": "job-search-card__location"})
    for loc in location:
        locations.append(loc.text.replace("\n", " ").strip())

    # Title
    title = soup.findAll("h3", {"class": "base-search-card__title"})
    for t in title:
        titles.append(t.text.replace("\n", " ").strip())

    # Country
    for c in location:
        countries.append(c.text.replace("\n", " ").strip().split(',')[-1])

    # Job links
    job_links = soup.findAll("a", {"class": "base-card__full-link"})
    for link in job_links:
        links.append(link["href"])

    # Company names
    names = soup.findAll("a", {"class": "hidden-nested-link"})
    for name in names:
        company_name.append(name.text.replace("\n", " ").strip())

# Keywords
desc_keywords = ['responsibilities', 'responsibilities:', 'what you’ll do', 'what you will do', 'deskripsi', 'tugas', 'role', 'job description:', 'job description', 
                 'your main duties in flying with us', 'your impact', "you’ll get to"]
req_keywords = ['requirement', 'requirement:', 'requirements', 'requirements:', 'qualification', 'qualification:', 'qualifications', 'qualifications:', 'kualifikasi',
                'what you’ll need', 'what you will need', 'you should have', 'skills', 'experience', 'job requirements', 'mandatory belongings that you must prepare', 
                'who we look for', "you’ll need to succeed", 'what this role needs', 'kriteria']

for link in links:
    try:
        job_page = requests.get(link)
        job_soup = BeautifulSoup(job_page.text, 'html.parser')

        # Extract work type
        work_type_found = "Not Found"
        preference_button = job_soup.find("button", class_="job-details-preferences-and-skills")
        if preference_button:
            pill_divs = preference_button.find_all("div", class_="job-details-preferences-and-skills__pill")
            for pill in pill_divs:
                span = pill.find("span", class_="ui-label ui-label--accent-3 text-body-small")
                if span and span.text.strip() in ["Remote", "On-site", "Hybrid"]:
                    work_type_found = span.text.strip()
                    break
        work_types.append(work_type_found)

        # Extract job description & requirements
        desc_block = job_soup.find("div", {"class": "show-more-less-html__markup"})
        if desc_block:
            items = desc_block.find_all("li")
            li_texts = [li.text.strip() for li in items]

            desc_part = []
            req_part = []

            for li_text in li_texts:
                text_lower = li_text.lower()
                
                if any(k in text_lower for k in req_keywords):
                    req_part.append(li_text)
                elif any(k in text_lower for k in desc_keywords):
                    desc_part.append(li_text)
                else:
                    if any(word in text_lower for word in ['analyze', 'develop', 'manage', 'design', 'plan']):
                        desc_part.append(li_text)
                    else:
                        req_part.append(li_text)

            job_description.append(" | ".join(desc_part) if desc_part else "Not Found")
            job_requirement.append(" | ".join(req_part) if req_part else "Not Found")
        else:
            job_description.append("Not Found")
            job_requirement.append("Not Found")

        time.sleep(1)
    except Exception as e:
        job_description.append("Error: " + str(e))
        job_requirement.append("Error: " + str(e))
        work_types.append("Error")

In [30]:
df8 = pd.DataFrame({
    "Title": titles,
    "Company": company_name,
    "Location": locations,
    "Country": countries,
    "Job Description": job_description,
    "Job Requirements": job_requirement,
    "Work Type" : work_types,
    "Link": links
})

df8

,Title,Company,Location,Country,Job Description,Job Requirements,Work Type,Link
0,DevOps,PT. RAKHASA ARTHA WISESA,Jakarta Metropolitan Area,Jakarta Metropolitan Area,Pengalaman dalam pemantauan dan log management...,Pendidikan | Minimal lulusan S1 di bidang Ilmu...,Not Found,https://id.linkedin.com/jobs/view/devops-at-pt...
1,Devops,Binar Academy,"Jakarta, Jakarta, Indonesia",Indonesia,Not Found,Hybrid (sebagian besar di Jabodetabek area) | ...,Not Found,https://id.linkedin.com/jobs/view/devops-at-bi...
2,DevSecOps,PT ITSEC Asia Tbk,"Jakarta, Jakarta, Indonesia",Indonesia,"Design, implement, and maintain secure CI/CD p...",Not Found,Not Found,https://id.linkedin.com/jobs/view/devsecops-at...
3,Devops Engineering,bitcorp. (PT. Bringin Inti Teknologi),"Jakarta, Jakarta, Indonesia",Indonesia,Automate infrastructure management using tools...,"Build, maintain, and optimize Continuous Integ...",Not Found,https://id.linkedin.com/jobs/view/devops-engin...
4,Sr DevOps,Sprout Digital Labs,"Kota Tangerang Selatan, Banten, Indonesia",Indonesia,Building and setting up new development tools ...,Defining/Managing/Monitoring production infras...,Not Found,https://id.linkedin.com/jobs/view/sr-devops-at...
...,...,...,...,...,...,...,...,...
235,Site Reliability Engineer,PT Astra International Tbk,"Jakarta, Jakarta, Indonesia",Indonesia,Menjembatani kesenjangan antara Tim Core Infra...,Menguasai Ketersediaan dan Performa aplikasi d...,Not Found,https://id.linkedin.com/jobs/view/site-reliabi...
236,DevOps Engineer,Sprint Asia,"Jakarta, Jakarta, Indonesia",Indonesia,Design and implement cluster HCI infrastructur...,Perform corrective and preventive maintenance ...,Not Found,https://id.linkedin.com/jobs/view/devops-engin...
237,Site Reliability Engineer (SRE),Fazz,"Jakarta, Jakarta, Indonesia",Indonesia,"Deploy, automate, maintain, and manage various...",Understanding the high-level overview of our a...,Not Found,https://id.linkedin.com/jobs/view/site-reliabi...
238,Country Head - Indonesia,Searce Inc,"Jakarta, Indonesia",Indonesia,Oversee and manage a team of Regional Sales Ma...,An enthusiastic Technology Consulting & Busine...,Not Found,https://id.linkedin.com/jobs/view/country-head...


# Writer

In [31]:
url_writer = 'https://www.linkedin.com/jobs/search/?currentJobId=4219587624&geoId=102478259&keywords=writer&refresh=true'

In [32]:
locations = []
titles = []
countries = []    
links = []
company_name = []
job_description = []
job_requirement = []
work_types = []

for i in range(0, 100, 25):
    url = url_writer + "&start=" + str(i)
    request = requests.get(url)
    soup = BeautifulSoup(request.text, 'html.parser')

    # Location
    location = soup.findAll("span", {"class": "job-search-card__location"})
    for loc in location:
        locations.append(loc.text.replace("\n", " ").strip())

    # Title
    title = soup.findAll("h3", {"class": "base-search-card__title"})
    for t in title:
        titles.append(t.text.replace("\n", " ").strip())

    # Country
    for c in location:
        countries.append(c.text.replace("\n", " ").strip().split(',')[-1])

    # Job links
    job_links = soup.findAll("a", {"class": "base-card__full-link"})
    for link in job_links:
        links.append(link["href"])

    # Company names
    names = soup.findAll("a", {"class": "hidden-nested-link"})
    for name in names:
        company_name.append(name.text.replace("\n", " ").strip())

# Keywords
desc_keywords = ['responsibilities', 'responsibilities:', 'what you’ll do', 'what you will do', 'deskripsi', 'tugas', 'role', 'job description:', 'job description', 
                 'your main duties in flying with us', 'your impact', "you’ll get to"]
req_keywords = ['requirement', 'requirement:', 'requirements', 'requirements:', 'qualification', 'qualification:', 'qualifications', 'qualifications:', 'kualifikasi',
                'what you’ll need', 'what you will need', 'you should have', 'skills', 'experience', 'job requirements', 'mandatory belongings that you must prepare', 
                'who we look for', "you’ll need to succeed", 'what this role needs', 'kriteria']

for link in links:
    try:
        job_page = requests.get(link)
        job_soup = BeautifulSoup(job_page.text, 'html.parser')

        # Extract work type
        work_type_found = "Not Found"
        preference_button = job_soup.find("button", class_="job-details-preferences-and-skills")
        if preference_button:
            pill_divs = preference_button.find_all("div", class_="job-details-preferences-and-skills__pill")
            for pill in pill_divs:
                span = pill.find("span", class_="ui-label ui-label--accent-3 text-body-small")
                if span and span.text.strip() in ["Remote", "On-site", "Hybrid"]:
                    work_type_found = span.text.strip()
                    break
        work_types.append(work_type_found)

        # Extract job description & requirements
        desc_block = job_soup.find("div", {"class": "show-more-less-html__markup"})
        if desc_block:
            items = desc_block.find_all("li")
            li_texts = [li.text.strip() for li in items]

            desc_part = []
            req_part = []

            for li_text in li_texts:
                text_lower = li_text.lower()
                
                if any(k in text_lower for k in req_keywords):
                    req_part.append(li_text)
                elif any(k in text_lower for k in desc_keywords):
                    desc_part.append(li_text)
                else:
                    if any(word in text_lower for word in ['analyze', 'develop', 'manage', 'design', 'plan']):
                        desc_part.append(li_text)
                    else:
                        req_part.append(li_text)

            job_description.append(" | ".join(desc_part) if desc_part else "Not Found")
            job_requirement.append(" | ".join(req_part) if req_part else "Not Found")
        else:
            job_description.append("Not Found")
            job_requirement.append("Not Found")

        time.sleep(1)
    except Exception as e:
        job_description.append("Error: " + str(e))
        job_requirement.append("Error: " + str(e))
        work_types.append("Error")

In [33]:
df9 = pd.DataFrame({
    "Title": titles,
    "Company": company_name,
    "Location": locations,
    "Country": countries,
    "Job Description": job_description,
    "Job Requirements": job_requirement,
    "Work Type" : work_types,
    "Link": links
})

df9

,Title,Company,Location,Country,Job Description,Job Requirements,Work Type,Link
0,Creative Copywriter Internship,PT Bank Sinarmas Tbk,"Jakarta, Indonesia",Indonesia,Not Found,Not Found,Not Found,https://id.linkedin.com/jobs/view/creative-cop...
1,Freelance Proofreader,Firstsource,Indonesia,Indonesia,Not Found,Competitive pay (up to ₹1000/hour). | Flexible...,Not Found,https://id.linkedin.com/jobs/view/freelance-pr...
2,Penulis Lepas Bahasa Indonesia,Outlier,Indonesia,Indonesia,Not Found,Membaca teks berbahasa Indonesia untuk memberi...,Not Found,https://id.linkedin.com/jobs/view/penulis-lepa...
3,Copywriter - Contract-based,PT Astra International Tbk,"Jakarta, Jakarta, Indonesia",Indonesia,Not Found,Menulis teks iklan yang menarik dan persuasif ...,Not Found,https://id.linkedin.com/jobs/view/copywriter-c...
4,Copywriter,dentsu,"Jakarta, Jakarta, Indonesia",Indonesia,"Collaborative team player, working effectively...","Write narratives, scripts, storyboards, annota...",Not Found,https://id.linkedin.com/jobs/view/copywriter-a...
...,...,...,...,...,...,...,...,...
175,"Sosial Media Project, Script Writer Freelance ...",State University of Malang,"Malang, East Java, Indonesia",Indonesia,Not Found,Not Found,Not Found,https://id.linkedin.com/jobs/view/sosial-media...
176,Copywriter,RiDiK (a Subsidiary of CLPS. Nasdaq: CLPS),"Jakarta, Indonesia",Indonesia,"Develop captions, short stories, and copy that...",Write captivating content for social media pla...,Not Found,https://id.linkedin.com/jobs/view/copywriter-a...
177,Technical Writer,PT. Metrodata Electronics Tbk.,"Jakarta, Jakarta, Indonesia",Indonesia,"Write easy-to-understand user interface text, ...",Work with internal teams to obtain an in-depth...,Not Found,https://id.linkedin.com/jobs/view/technical-wr...
178,Suomenkielinen freelance-kirjoittaja,Outlier,Indonesia,Indonesia,Not Found,Suomenkielisten tekstien lukeminen ja tekoälym...,Not Found,https://id.linkedin.com/jobs/view/suomenkielin...


# Editor

In [34]:
url_editor = 'https://www.linkedin.com/jobs/search/?currentJobId=4229431159&geoId=102478259&keywords=editor&refresh=true'

In [35]:
locations = []
titles = []
countries = []    
links = []
company_name = []
job_description = []
job_requirement = []
work_types = []

for i in range(0, 100, 25):
    url = url_editor + "&start=" + str(i)
    request = requests.get(url)
    soup = BeautifulSoup(request.text, 'html.parser')

    # Location
    location = soup.findAll("span", {"class": "job-search-card__location"})
    for loc in location:
        locations.append(loc.text.replace("\n", " ").strip())

    # Title
    title = soup.findAll("h3", {"class": "base-search-card__title"})
    for t in title:
        titles.append(t.text.replace("\n", " ").strip())

    # Country
    for c in location:
        countries.append(c.text.replace("\n", " ").strip().split(',')[-1])

    # Job links
    job_links = soup.findAll("a", {"class": "base-card__full-link"})
    for link in job_links:
        links.append(link["href"])

    # Company names
    names = soup.findAll("a", {"class": "hidden-nested-link"})
    for name in names:
        company_name.append(name.text.replace("\n", " ").strip())

# Keywords
desc_keywords = ['responsibilities', 'responsibilities:', 'what you’ll do', 'what you will do', 'deskripsi', 'tugas', 'role', 'job description:', 'job description', 
                 'your main duties in flying with us', 'your impact', "you’ll get to"]
req_keywords = ['requirement', 'requirement:', 'requirements', 'requirements:', 'qualification', 'qualification:', 'qualifications', 'qualifications:', 'kualifikasi',
                'what you’ll need', 'what you will need', 'you should have', 'skills', 'experience', 'job requirements', 'mandatory belongings that you must prepare', 
                'who we look for', "you’ll need to succeed", 'what this role needs', 'kriteria']

for link in links:
    try:
        job_page = requests.get(link)
        job_soup = BeautifulSoup(job_page.text, 'html.parser')

        # Extract work type
        work_type_found = "Not Found"
        preference_button = job_soup.find("button", class_="job-details-preferences-and-skills")
        if preference_button:
            pill_divs = preference_button.find_all("div", class_="job-details-preferences-and-skills__pill")
            for pill in pill_divs:
                span = pill.find("span", class_="ui-label ui-label--accent-3 text-body-small")
                if span and span.text.strip() in ["Remote", "On-site", "Hybrid"]:
                    work_type_found = span.text.strip()
                    break
        work_types.append(work_type_found)

        # Extract job description & requirements
        desc_block = job_soup.find("div", {"class": "show-more-less-html__markup"})
        if desc_block:
            items = desc_block.find_all("li")
            li_texts = [li.text.strip() for li in items]

            desc_part = []
            req_part = []

            for li_text in li_texts:
                text_lower = li_text.lower()
                
                if any(k in text_lower for k in req_keywords):
                    req_part.append(li_text)
                elif any(k in text_lower for k in desc_keywords):
                    desc_part.append(li_text)
                else:
                    if any(word in text_lower for word in ['analyze', 'develop', 'manage', 'design', 'plan']):
                        desc_part.append(li_text)
                    else:
                        req_part.append(li_text)

            job_description.append(" | ".join(desc_part) if desc_part else "Not Found")
            job_requirement.append(" | ".join(req_part) if req_part else "Not Found")
        else:
            job_description.append("Not Found")
            job_requirement.append("Not Found")

        time.sleep(1)
    except Exception as e:
        job_description.append("Error: " + str(e))
        job_requirement.append("Error: " + str(e))
        work_types.append("Error")

In [36]:
df10 = pd.DataFrame({
    "Title": titles,
    "Company": company_name,
    "Location": locations,
    "Country": countries,
    "Job Description": job_description,
    "Job Requirements": job_requirement,
    "Work Type" : work_types,
    "Link": links
})

df10

,Title,Company,Location,Country,Job Description,Job Requirements,Work Type,Link
0,Penulis Lepas Bahasa Indonesia,Outlier,Indonesia,Indonesia,Not Found,Membaca teks berbahasa Indonesia untuk memberi...,Not Found,https://id.linkedin.com/jobs/view/penulis-lepa...
1,Freelance Proofreader,Firstsource,Indonesia,Indonesia,Not Found,Competitive pay (up to ₹1000/hour). | Flexible...,Not Found,https://id.linkedin.com/jobs/view/freelance-pr...
2,Editor Konten Bilingual,DataAnnotation,Indonesia,Indonesia,Not Found,Not Found,Not Found,https://id.linkedin.com/jobs/view/editor-konte...
3,Arabic (Saudi) Freelance Writer,Outlier,Indonesia,Indonesia,Not Found,Outlier is looking for talented writers with f...,Not Found,https://id.linkedin.com/jobs/view/arabic-saudi...
4,Scrittore freelance italiano,Outlier,Indonesia,Indonesia,Not Found,Leggendo testi in italiano per classificare un...,Not Found,https://id.linkedin.com/jobs/view/scrittore-fr...
...,...,...,...,...,...,...,...,...
235,Photo Editor - Holiday Shift,Pippeline,"Kebon Jeruk, Jakarta, Indonesia",Indonesia,"Editor menjadi bagian tim image editing, yang ...",Memiliki pola pikir berkembang atau Growth Min...,Not Found,https://id.linkedin.com/jobs/view/photo-editor...
236,Video Editor,Soulshine Bali,"Gianyar, Bali, Indonesia",Indonesia,Not Found,Having minimum 2-year experience in the same f...,Not Found,https://id.linkedin.com/jobs/view/video-editor...
237,Content Writer,cmlabs,"Malang, East Java, Indonesia",Indonesia,Job Description | An understanding of how cont...,Company Description | Qualification | Skills |...,Not Found,https://id.linkedin.com/jobs/view/content-writ...
238,Content Writer,cmlabs,"Malang, East Java, Indonesia",Indonesia,Job Description | An understanding of how cont...,Company Description | Qualification | Skills |...,Not Found,https://id.linkedin.com/jobs/view/content-writ...


# Combine all df

In [37]:
# Gabungkan semua DataFrame
combined_df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10], ignore_index=True)

# Simpan ke file CSV
combined_df.to_csv("jobs.csv", index=False)